In [1]:
from datetime import date

import pandas as pd
import yaml

In [2]:
from sqlalchemy import create_engine

with open('./config.yml', 'r') as f:
        config = yaml.safe_load(f)
        config_ryf = config['RAPIDO_Y_FURIOSO']
        config_etl = config['ETL']

url_ryf = (f"{config_ryf['drivername']}://{config_ryf['user']}:{config_ryf['password']}@{config_ryf['host']}:"
        f"{config_ryf['port']}/{config_ryf['dbname']}")
url_etl = (f"{config_etl['drivername']}://{config_etl['user']}:{config_etl['password']}@{config_etl['host']}:"
        f"{config_etl['port']}/{config_etl['dbname']}")

ryf = create_engine(url_ryf)
etl_conn = create_engine(url_etl)

In [3]:
dim_estado = pd.read_sql_table('mensajeria_estado', url_ryf)
dim_estado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           6 non-null      int64 
 1   nombre       6 non-null      object
 2   descripcion  6 non-null      object
dtypes: int64(1), object(2)
memory usage: 272.0+ bytes


In [4]:
dim_estado = dim_estado.rename(columns={
    'id': 'estado_id',
})

In [5]:
dim_estado.head()

,estado_id,nombre,descripcion
0,4,Recogido por mensajero,Recogido por mensajero
1,5,Entregado en destino,Entregado en destino
2,3,Con novedad,Tiene novedad
3,6,Terminado completo,Terminado completo
4,1,Iniciado,Creado por el usuario


In [6]:
dim_estado.to_sql('dim_estado', etl_conn, if_exists='replace',index_label='key_dim_estado')

6